## Initialization

In [1]:
from huggingface_hub import whoami

c:\ProgramData\miniconda3\envs\lerobot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check if you're logged in
info = whoami()
HF_USER = info['name']
print(f"✅ You are logged in as {HF_USER}")

✅ You are logged in as Unseq


## Teleop

In [ ]:
import subprocess
import signal
import os

def run_in_terminal(cmd, timeout=600, verbose=False):
    if verbose:
        print(f"Running command: {cmd}")
    process = subprocess.Popen(cmd)
    try:
        process.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        print(f"Command timed out after {timeout} seconds, process still running. Make sure to close the terminal.")


In [ ]:
robot_port = "COM11"
robot_id = "unseq_follower"
teleop_port = "COM10"
teleop_id = "unseq_leader"
teleop_duration = "10"
FPS = "30"

In [ ]:
teleoperate_no_cam = [
    "python", "-m", "lerobot.teleoperate",
    "--robot.type=so101_follower",
    f"--robot.port={robot_port}",
    f"--robot.id={robot_id}",
    "--teleop.type=so101_leader",
    f"--teleop.port={teleop_port}",
    f"--teleop.id={teleop_id}",
    f"--teleop_time_s={teleop_duration}",
]

run_in_terminal(teleoperate_no_cam)

Command timed out after 5 seconds, process still running. Make sure to close the terminal.


In [ ]:
FPS = 30
robot_cameras = (
	'{ front: {type: opencv, index_or_path: 2, width: 640, height: 480, fps: ' + str(FPS) + '}, '
	'top: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: ' + str(FPS) + '} }'
)

teleoperate_with_cam = [
	"python", "-m", "lerobot.teleoperate",
	"--robot.type=so101_follower",
	f"--robot.port={robot_port}",
	f"--robot.id={robot_id}",
	f"--robot.cameras={robot_cameras}",
	"--teleop.type=so101_leader",
	f"--teleop.port={teleop_port}",
	f"--teleop.id={teleop_id}",
	"--display_data=true",
	f"--teleop_time_s={teleop_duration}"
]

run_in_terminal(teleoperate_with_cam)

Command timed out after 10 seconds, process still running. Make sure to close the terminal.


In [ ]:
record_cmd = [
    "python", "-m", "lerobot.record",
    "--robot.type=so101_follower",
    "--robot.port=COM11",
    "--robot.id=unseq_follower",
    '--robot.cameras={ front: {type: opencv, index_or_path: 2, width: 640, height: 480, fps: 15}, top: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: 15}}',
    "--teleop.type=so101_leader",
    "--teleop.port=COM10",
    "--teleop.id=unseq_leader",
    "--display_data=true",
    f"--dataset.repo_id={HF_USER}/record-test3",
    "--dataset.num_episodes=2",
    '--dataset.single_task=Move object from blue to white square'
]

run_in_terminal(record_cmd, timeout=60)

python -m lerobot.replay --robot.type=so101_follower --robot.port=COM11 --robot.id=unseq_follower --dataset.repo_id=${HF_USER}/record-test3 --dataset.episode=1

how to remove episodes from dataset: https://github.com/huggingface/lerobot/pull/831 not yet merged, as the python script is not available in the repo.





## Train

train_cmd = [
    "python", "lerobot/scripts/train.py",
    f"--dataset.repo_id={HF_USER}/record-test-5",
    "--policy.type=act",
    "--output_dir=outputs/train/act_so101_test",
    "--job_name=act_so101_test",
    "--policy.device=cuda",
    "--wandb.enable=true"
]

run_in_terminal(train_cmd)

In [ ]:
import torch

# Check if CUDA is available
torch.cuda.is_available()


True

In [2]:
import torch
from torchcodec.codecs import Encodec

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create Encodec model
model = Encodec("encodec_24khz").to(device)
model.eval()

# Dummy audio: batch of 1, 1 channel, 10,000 samples (about 0.4 sec at 24 kHz)
waveform = torch.randn(1, 1, 10000).to(device)

# Forward pass: encode and decode
with torch.no_grad():
    encoded_frames = model.encode(waveform)
    decoded_waveform = model.decode(encoded_frames)

print(f"Original shape: {waveform.shape}")
print(f"Decoded shape: {decoded_waveform.shape}")

Stay tuned...


ModuleNotFoundError: No module named 'torchcodec.codecs'

In [ ]:
!python 

## Evaluating trained model

In [24]:
!python lerobot/scripts/eval.py \
    --policy.path=lerobot/diffusion_pusht \
    --env.type=pusht \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --use_amp=false \
    --device=cuda

python: can't open file 'c:\\Users\\jprij\\Documents\\lerobot\\lerobot\\scripts\\lerobot\\scripts\\eval.py': [Errno 2] No such file or directory


# Debugging

In [ ]:
import cv2
import os
import time
from pathlib import Path

# Open default camera (index 0)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Cannot open camera at index 0")
else:
    # Warm up the camera by waiting and reading a few frames
    time.sleep(2)  # Wait 2 seconds

    for i in range(5):  # Discard first few frames
        cap.read()

    ret, frame = cap.read()
    if not ret or frame is None:
        print("Camera opened but failed to read a frame")
    else:
        print(f"Successfully captured frame of shape: {frame.shape}")
        
        # Get path to Downloads folder
        downloads_path = str(Path.home() / "Downloads")
        filename = "captured_image.jpg"
        save_path = os.path.join(downloads_path, filename)

        # Save the frame as JPEG
        cv2.imwrite(save_path, frame)
        print(f"📸 Image saved to: {save_path}")

    cap.release()


In [ ]:
# in common/cameras/opencv/camera_opencv
self.videocapture = cv2.VideoCapture(self.index_or_path, self.backend) 
# to
self.videocapture = cv2.VideoCapture(self.index_or_path)#, self.backend)

In [ ]:
from lerobot.common.cameras.opencv.configuration_opencv import OpenCVCameraConfig
from lerobot.common.cameras.opencv.camera_opencv import OpenCVCamera
from lerobot.common.cameras.configs import ColorMode, Cv2Rotation

# Construct an `OpenCVCameraConfig` with your desired FPS, resolution, color mode, and rotation.
config = OpenCVCameraConfig(
    index_or_path=2,
    fps=30,
    width=640,
    height=480,
    color_mode=ColorMode.RGB,
    rotation=Cv2Rotation.NO_ROTATION
)

# Instantiate and connect an `OpenCVCamera`, performing a warm-up read (default).
camera = OpenCVCamera(config)
camera.connect()

# Read frames asynchronously in a loop via `async_read(timeout_ms)`
try:
    t = time.time()
    for i in range(10):
        frame = camera.async_read(timeout_ms=2000) #NOTE: at least 2 seconds timeout, otherwise it will fail
        print(f"Async frame {i} shape:", frame.shape)
        
        try: 
            print(1/(time.time() - t), 'FPS')
        except ZeroDivisionError:
            pass
        t = time.time()
finally:
    camera.disconnect()

In [2]:
# lessons
- webcam USB on dock will cap at 1 FPS (at least with my dock and everything connected) -> give each camera own port.
- timeout of our USB webcams must be at least 2 seconds.
- @ 1280x720, 7-8 FPS can be reached with USB webcam.
- @ 640x480, 30 FPS can be reached with USB webcam (in 2 of the ports, other port is capped at 15 FPS.)

SyntaxError: invalid decimal literal (2432197884.py, line 4)